<a href="https://colab.research.google.com/github/SDCoulter/big_data_nlp/blob/main/10_PySpark_ETL1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Latest version of Spark.
spark_version = 'spark-3.1.2/'
os.environ['SPARK_VERSION'] = spark_version

# Install Spark and Java.
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

# Set environment variables.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-3.1.2-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [61.8 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [2]:
# Download a Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-06-09 00:09:25--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.61MB/s    in 0.2s    

2021-06-09 00:09:25 (5.61 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
# Start Spark session, with additional config for Postgres.
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
# Import SparkFiles and extract data from S3 Buckets to DFs.
from pyspark import SparkFiles
url1 = "https://sdcoulter-bucket.s3.amazonaws.com/user_data.csv"
url2 = "https://sdcoulter-bucket.s3.amazonaws.com/user_payment.csv"
spark.sparkContext.addFile(url1)
spark.sparkContext.addFile(url2)
user_data_df = spark.read.csv(SparkFiles.get("user_data.csv"), sep=",", header=True)
user_payment_df = spark.read.csv(SparkFiles.get("user_payment.csv"), sep=",", header=True)

In [7]:
# User Data DF.
user_data_df.show(5)

+---+----------+---------+-----------+-------------------+--------------+---------+
| id|first_name|last_name|active_user|     street_address|         state| username|
+---+----------+---------+-----------+-------------------+--------------+---------+
|  1|    Cletus|  Lithcow|      FALSE|78309 Riverside Way|      Virginia|ibearham0|
|  2|       Caz|   Felgat|      FALSE|83 Hazelcrest Place|       Alabama| wwaller1|
|  3|     Kerri|  Crowson|      FALSE|     112 Eliot Pass|North Carolina|ichesnut2|
|  4|   Freddie|    Caghy|      FALSE|    15 Merchant Way|      New York|  tsnarr3|
|  5|   Sadella|    Deuss|      FALSE|   079 Acker Avenue|     Tennessee|fwherrit4|
+---+----------+---------+-----------+-------------------+--------------+---------+
only showing top 5 rows



In [8]:
# User Payment DF.
user_payment_df.show(5)

+----------+---------+--------------------+
|billing_id| username|        cc_encrypted|
+----------+---------+--------------------+
|         1|ibearham0|a799fcafe47d7fb19...|
|         2| wwaller1|a799fcafe47d7fb19...|
|         3|ichesnut2|a799fcafe47d7fb19...|
|         4|  tsnarr3|a799fcafe47d7fb19...|
|         5|fwherrit4|a799fcafe47d7fb19...|
+----------+---------+--------------------+
only showing top 5 rows



In [9]:
# Join the two tables.
joined_df = user_data_df.join(user_payment_df, on='username', how='inner')
joined_df.show(5)

+---------+---+----------+---------+-----------+-------------------+--------------+----------+--------------------+
| username| id|first_name|last_name|active_user|     street_address|         state|billing_id|        cc_encrypted|
+---------+---+----------+---------+-----------+-------------------+--------------+----------+--------------------+
|ibearham0|  1|    Cletus|  Lithcow|      FALSE|78309 Riverside Way|      Virginia|         1|a799fcafe47d7fb19...|
| wwaller1|  2|       Caz|   Felgat|      FALSE|83 Hazelcrest Place|       Alabama|         2|a799fcafe47d7fb19...|
|ichesnut2|  3|     Kerri|  Crowson|      FALSE|     112 Eliot Pass|North Carolina|         3|a799fcafe47d7fb19...|
|  tsnarr3|  4|   Freddie|    Caghy|      FALSE|    15 Merchant Way|      New York|         4|a799fcafe47d7fb19...|
|fwherrit4|  5|   Sadella|    Deuss|      FALSE|   079 Acker Avenue|     Tennessee|         5|a799fcafe47d7fb19...|
+---------+---+----------+---------+-----------+-------------------+----

In [10]:
# Drop any rows with null/NaN values.
dropna_df = joined_df.dropna()
dropna_df.show(5)

+---------+---+----------+---------+-----------+-------------------+--------------+----------+--------------------+
| username| id|first_name|last_name|active_user|     street_address|         state|billing_id|        cc_encrypted|
+---------+---+----------+---------+-----------+-------------------+--------------+----------+--------------------+
|ibearham0|  1|    Cletus|  Lithcow|      FALSE|78309 Riverside Way|      Virginia|         1|a799fcafe47d7fb19...|
| wwaller1|  2|       Caz|   Felgat|      FALSE|83 Hazelcrest Place|       Alabama|         2|a799fcafe47d7fb19...|
|ichesnut2|  3|     Kerri|  Crowson|      FALSE|     112 Eliot Pass|North Carolina|         3|a799fcafe47d7fb19...|
|  tsnarr3|  4|   Freddie|    Caghy|      FALSE|    15 Merchant Way|      New York|         4|a799fcafe47d7fb19...|
|fwherrit4|  5|   Sadella|    Deuss|      FALSE|   079 Acker Avenue|     Tennessee|         5|a799fcafe47d7fb19...|
+---------+---+----------+---------+-----------+-------------------+----

In [11]:
# Filter for active users.
from pyspark.sql.functions import col
cleaned_df = dropna_df.filter(col("active_user") == True)
cleaned_df.show(5)

+------------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|    username| id|first_name|last_name|active_user|      street_address|               state|billing_id|        cc_encrypted|
+------------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|  fstappard5|  6|    Fraser|  Korneev|       TRUE|  76084 Novick Court|           Minnesota|         6|a799fcafe47d7fb19...|
|  lhambling6|  7|    Demott|   Rapson|       TRUE|    86320 Dahle Park|District of Columbia|         7|a799fcafe47d7fb19...|
|   wheinerte| 15|   Sadella|    Jaram|       TRUE|7528 Waxwing Terrace|         Connecticut|        15|a799fcafe47d7fb19...|
|droughsedgeg| 17|    Hewitt|  Trammel|       TRUE|    2455 Corry Alley|      North Carolina|        17|a799fcafe47d7fb19...|
|   ydudeniei| 19|       Ted|  Knowlys|       TRUE|      31 South Drive|                Ohio|        19|a799fcafe47d7f

In [12]:
# Create user DF to match the active_user table.
clean_user_df = cleaned_df.select(['id', 'first_name', 'last_name', 'username'])
clean_user_df.show(5)

+---+----------+---------+------------+
| id|first_name|last_name|    username|
+---+----------+---------+------------+
|  6|    Fraser|  Korneev|  fstappard5|
|  7|    Demott|   Rapson|  lhambling6|
| 15|   Sadella|    Jaram|   wheinerte|
| 17|    Hewitt|  Trammel|droughsedgeg|
| 19|       Ted|  Knowlys|   ydudeniei|
+---+----------+---------+------------+
only showing top 5 rows



In [13]:
# Create the DF to match the billing_info table.
clean_billing_df = cleaned_df.select(['billing_id', 'street_address', 'state', 'username'])
clean_billing_df.show(5)

+----------+--------------------+--------------------+------------+
|billing_id|      street_address|               state|    username|
+----------+--------------------+--------------------+------------+
|         6|  76084 Novick Court|           Minnesota|  fstappard5|
|         7|    86320 Dahle Park|District of Columbia|  lhambling6|
|        15|7528 Waxwing Terrace|         Connecticut|   wheinerte|
|        17|    2455 Corry Alley|      North Carolina|droughsedgeg|
|        19|      31 South Drive|                Ohio|   ydudeniei|
+----------+--------------------+--------------------+------------+
only showing top 5 rows



In [14]:
# Create the DF to match the payment_info table.
clean_payment_df = cleaned_df.select(['billing_id', 'cc_encrypted'])
clean_payment_df.show(5)

+----------+--------------------+
|billing_id|        cc_encrypted|
+----------+--------------------+
|         6|a799fcafe47d7fb19...|
|         7|a799fcafe47d7fb19...|
|        15|a799fcafe47d7fb19...|
|        17|a799fcafe47d7fb19...|
|        19|a799fcafe47d7fb19...|
+----------+--------------------+
only showing top 5 rows

